In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch



In [ ]:
import torch
import torchvision
from torch.utils.data import Dataset, Subset
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
# Load all the images
transformation = transforms.Compose([
    #resize to a specific size
    transforms.Resize((224,224)),
    # Randomly augment the image data
        # Random horizontal flip
     transforms.RandomHorizontalFlip(0.5),
#         # Random vertical flip
     transforms.RandomVerticalFlip(0.3),
    # transform to tensors
    transforms.ToTensor(),
    # Normalize the pixel values (in R, G, and B channels)
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load all of the images, transforming them
dataset = torchvision.datasets.ImageFolder(
    root='/kaggle/input/iith-dl-contest-2024/train/train',
    transform=transformation,
    #target_transform = class_to_idx
)
# diving train data into train and validation
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.1, random_state=42)

train_dataset = Subset(dataset,train_indices) 
val_dataset =  Subset(dataset,val_indices)

# define a loader for the training data we can iterate through in 50-image batches
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle = True

)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=64,
    shuffle = True

)

In [ ]:
import os
import torch
import torchvision
from torch.utils.data import Dataset
import torchvision.transforms as transforms


cls_dir='/kaggle/input/iith-dl-contest-2024/test/test'
filenames = sorted(os.listdir(cls_dir))


# Usage example
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    
])

test_dataset = torchvision.datasets.ImageFolder(root='/kaggle/input/iith-dl-contest-2024/test', transform=transform)
 
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,

)


In [ ]:
print(dataset.classes)

In [ ]:
!pip install torchensemble

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader
import torchensemble as te
import timm

num_classes = len(dataset.classes)
base =  timm.create_model("resnet34",pretrained=False,num_classes=num_classes)
model = te.gradient_boosting.GradientBoostingClassifier(estimator = base, n_estimators = 4,cuda=True)

model.set_optimizer('Adam',             # parameter optimizer
                    lr=1e-3,            # learning rate of the optimizer
                    weight_decay=5e-4)  # weight decay of the optimizer

model.set_criterion(nn.CrossEntropyLoss())

In [ ]:
#model.fit(train_loader = train_loader, epochs = 10,save_model = True)

In [ ]:
from torchensemble.utils import io
io.load(model, '/kaggle/input/ensemble-4')
acc =model.evaluate(val_loader,return_loss=False)
acc

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import pandas as pd
csv_file = "output.csv"
column = ["ID", "Category"]
df = pd.DataFrame(columns=column)
csv_file = "output.csv"

j = 0
for images, labels in test_loader:
    images = images.to(device)
    y = model.forward(images)
    _,pred = torch.max(y,1)
    new_row = pd.DataFrame({"ID":[filenames[i] for i in range(j,j+images.shape[0])],"Category":[dataset.classes[p] for p in pred]})
    df = pd.concat([df,new_row], ignore_index=True)
    j = j + images.shape[0]
    del images,y
df.to_csv(csv_file, index=False)